#DataFrame Column Class

** Data Source **
* One hour of Pagecounts from the English Wikimedia projects captured August 5, 2016, at 12:00 PM UTC.
* Size on Disk: ~23 MB
* Type: Compressed Parquet File
* More Info: <a href="https://dumps.wikimedia.org/other/pagecounts-raw" target="_blank">Page view statistics for Wikimedia projects</a>

**Technical Accomplishments:**
* Continue exploring the `DataFrame` set of APIs.
* Introduce the `Column` class

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) **The Data Source**

We will be using the same data source as our previous notebook.

As such, we can go ahead and start by creating our initial `DataFrame`.

In [0]:
#(source, sasEntity, sasToken) = getAzureDataSource()
#spark.conf.set(sasEntity, sasToken)

parquetFile = "wasbs://spark-ui-simulator@dbacademy.blob.core.windows.net" + "/wikipedia/pagecounts/staging_parquet_en_only_clean/"

In [0]:
pagecountsEnAllDF = (spark  # Our SparkSession & Entry Point
  .read                     # Our DataFrameReader
  .parquet(parquetFile)     # Returns an instance of DataFrame
  .cache()                  # cache the data
)
print(pagecountsEnAllDF)

DataFrame[project: string, article: string, requests: int, bytes_served: bigint]


Let's take another look at the number of records in our `DataFrame`

In [0]:
total = pagecountsEnAllDF.count()

print("Record Count: {0:,}".format( total ))

Record Count: 2,345,943


Now let's take another peek at our data...

In [0]:
display(pagecountsEnAllDF)

As we view the data, we can see that there is no real rhyme or reason as to how the data is sorted.
* We cannot even tell if the column **project** is sorted - we are seeing only the first 1,000 of some 2.3 million records.
* The column **article** is not sorted as evident by the article **A_Little_Boy_Lost** appearing between a bunch of articles starting with numbers and symbols.
* The column **requests** is clearly not sorted.
* And our **bytes_served** contains nothing but zeros.

So let's start by sorting our data. In doing this, we can answer the following question:

What are the top 10 most requested articles?

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) orderBy(..) & sort(..)

If you look at the API docs, `orderBy(..)` is described like this:
> Returns a new Dataset sorted by the given expressions.

Both `orderBy(..)` and `sort(..)` arrange all the records in the `DataFrame` as specified.
* Like `distinct()` and `dropDuplicates()`, `sort(..)` and `orderBy(..)` are aliases for each other.
  * `sort(..)` appealing to functional programmers.
  * `orderBy(..)` appealing to developers with an SQL background.
* Like `orderBy(..)` there are two variants of these two methods:
  * `orderBy(Column)`
  * `orderBy(String)`
  * `sort(Column)`
  * `sort(String)`

All we need to do now is sort our previous `DataFrame`.

In [0]:
sortedDF = (pagecountsEnAllDF
  .orderBy("requests")
)
sortedDF.show(10, False)

+-------+----------------+--------+------------+
|project|article         |requests|bytes_served|
+-------+----------------+--------+------------+
|en     |!Ora_language   |1       |0           |
|en     |!Hukwe_language |1       |0           |
|en     |!?Revolution!?  |1       |0           |
|en     |!DOCTYPE        |1       |0           |
|en     |!Kung_San       |1       |0           |
|en     |!Ay,_caramba!   |1       |0           |
|en     |!Tre!           |1       |0           |
|en     |!T.O.O.H!       |1       |0           |
|en     |!O!kung_language|1       |0           |
|en     |!Gã!nge_language|1       |0           |
+-------+----------------+--------+------------+
only showing top 10 rows



As you can see, we are not sorting correctly.

We need to reverse the sort.

One might conclude that we could make a call like this:

`pagecountsEnAllDF.orderBy("requests desc")`

Try it in the cell below:

In [0]:
# Uncomment and try this:
pagecountsEnAllDF.orderBy("requests desc")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4170195121692590>, line 2
      1 # Uncomment and try this:
----> 2 pagecountsEnAllDF.orderBy("requests desc")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:2681, in DataFrame.sort(self, *cols, **kwargs)
   2590 def sort(
   2591     self, *cols: Union[str, Column, List[Union[str, Column]]], **kwargs: Any
   2592 ) -> "DataFrame":
   2593     """Returns a new :class:`DataFrame` sorted by the specified column(s).
   2594 
   2595     .. versionadde


Why does this not work?
* The `DataFrames` API is built upon an SQL engine.
* There is a lot of familiarity with this API and SQL syntax in general.
* The problem is that `orderBy(..)` expects the name of the column.
* What we specified was an SQL expression in the form of **requests desc**.
* What we need is a way to programmatically express such an expression.
* This leads us to the second variant, `orderBy(Column)` and more specifically, the class `Column`.

** *Note:* ** *Some of the calls in the `DataFrames` API actually accept SQL expressions.*<br/>
*While these functions will appear in the docs as `someFunc(String)` it's very*<br>
*important to thoroughly read and understand what the parameter actually represents.*

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) The Column Class

The `Column` class is an object that encompasses more than just the name of the column, but also column-level-transformations, such as sorting in a descending order.

The first question to ask is how do I create a `Column` object?

In Scala we have these options:

** *Note:* ** *We are showing both the Scala and Python versions below for comparison.*<br/>
*Make sure to run only the one cell for your notebook's default language (Scala or Python)*

In [0]:
%scala

// Scala & Python both support accessing a column from a known DataFrame
// Uncomment this if you are using the Scala version of this notebook
// val columnA = pagecountsEnAllDF("requests")    

// This option is Scala specific, but is arugably the cleanest and easy to read.
val columnB = $"requests"          

// If we import ...sql.functions, we get a couple of more options:
import org.apache.spark.sql.functions._

// This uses the col(..) function
val columnC = col("requests")

// This uses the expr(..) function which parses an SQL Expression
val columnD = expr("a + 1")

// This uses the lit(..) to create a literal (constant) value.
val columnE = lit("abc")

In Python we have these options:

In [0]:
%python

# Scala & Python both support accessing a column from a known DataFrame
# Uncomment this if you are using the Python version of this notebook
columnA = pagecountsEnAllDF["requests"]

# The $"column-name" version that works for Scala does not work in Python
# columnB = $"requests"      

# If we import ...sql.functions, we get a couple of more options:
from pyspark.sql.functions import *

# This uses the col(..) function
columnC = col("requests")

# This uses the expr(..) function which parses an SQL Expression
columnD = expr("a + 1")

# This uses the lit(..) to create a literal (constant) value.
columnE = lit("abc")

# Print the type of each attribute
print("columnC: {}".format(columnC))
print("columnD: {}".format(columnD))
print("columnE: {}".format(columnE))

columnC: Column<'requests'>
columnD: Column<'(a + 1)'>
columnE: Column<'abc'>



In the case of Scala, the cleanest version is the **$"column-name"** variant.

In the case of Python, the cleanest version is the **col("column-name")** variant.

So with that, we can now create a `Column` object, and apply the `desc()` operation to it:

** *Note:* ** *We are introducing `...sql.functions` specifically for creating `Column` objects.*<br/>
*We will be reviewing the multitude of other commands available from this part of the API in future notebooks.*

In [0]:
column = col("requests").desc()

# Print the column type
print("column:", column)

column: Column<'requests DESC NULLS LAST'>



And now we can piece it all together...

In [0]:
sortedDescDF = (pagecountsEnAllDF
  .orderBy( col("requests").desc() )
)  
sortedDescDF.show(10, False) # The top 10 is good enough for now

+-------+---------------------------+--------+------------+
|project|article                    |requests|bytes_served|
+-------+---------------------------+--------+------------+
|en     |Main_Page                  |865692  |0           |
|en.m   |Main_Page                  |176949  |0           |
|en     |Special:Search             |76231   |0           |
|en.m   |Donald_Trump               |59847   |0           |
|en     |Midas                      |55210   |0           |
|en     |Donald_Trump               |44640   |0           |
|en.m   |-                          |44130   |0           |
|en     |-                          |35663   |0           |
|en.m   |Melania_Trump              |24183   |0           |
|en     |Special:RecentChangesLinked|23419   |0           |
+-------+---------------------------+--------+------------+
only showing top 10 rows



It should be of no surprise that the **Main_Page** (in both the Wikipedia and Wikimedia projects) is the most requested page.

Followed shortly after that is **Special:Search**, Wikipedia's search page.

And if you consider that this data was captured in the August before the 2016 presidential election, the Trumps will be one of the most requested pages on Wikipedia.

### Review Column Class

The `Column` objects provide us a programmatic way to build up SQL-ish expressions.

Besides the `Column.desc()` operation we used above, we have a number of other operations that can be performed on a `Column` object.

Here is a preview of the various functions - we will cover many of these as we progress through the class:

**Column Functions**
* Various mathematical functions such as add, subtract, multiply & divide
* Various bitwise operators such as AND, OR & XOR
* Various null tests such as `isNull()`, `isNotNull()` & `isNaN()`.
* `as(..)`, `alias(..)` & `name(..)` - Returns this column aliased with a new name or names (in the case of expressions that return more than one column, such as explode).
* `between(..)` - A boolean expression that is evaluated to true if the value of this expression is between the given columns.
* `cast(..)` & `astype(..)` - Convert the column into type dataType.
* `asc(..)` - Returns a sort expression based on the ascending order of the given column name.
* `desc(..)` - Returns a sort expression based on the descending order of the given column name.
* `startswith(..)` - String starts with.
* `endswith(..)` - String ends with another string literal.
* `isin(..)` - A boolean expression that is evaluated to true if the value of this expression is contained by the evaluated values of the arguments.
* `like(..)` - SQL like expression
* `rlike(..)` - SQL RLIKE expression (LIKE with Regex).
* `substr(..)` - An expression that returns a substring.
* `when(..)` & `otherwise(..)` - Evaluates a list of conditions and returns one of multiple possible result expressions.

The complete list of functions differs from language to language.

## Next steps

Start the next lesson, [Work with Column expressions]($./2.DataFrame-Column-Expressions)